## Basic model

In [10]:
# from fastapi import FastAPI
# import ollama

# app = FastAPI()

# @app.post("/generate")
# def generate(prompt: str):
#     response = ollama.chat(model="deepseek-coder", messages=[{"role": "user", "content": prompt}])
#     return {"response": response['message']['content']}

# import nest_asyncio
# import uvicorn

# nest_asyncio.apply()  # Allows running uvicorn inside Jupyter
# uvicorn.run(app, host="0.0.0.0", port=8000)


In [11]:
# ! pip install ollama


## with mongodb integration

In [34]:
# from utils import mongo_module

# URI = mongo_module.mongo_creds_from_config()
# client = mongo_module.create_mongo_client(URI)

# if client:
#     sample_data = {"name": "John Doe", "age": 30, "city": "New York"}
#     mongo_module.insert_data(client, "testDB", "users", sample_data)

In [1]:
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
import ollama
import nest_asyncio
import uvicorn
from datetime import datetime
from pymongo import MongoClient
from utils import mongo_module

# MongoDB Setup
URI = mongo_module.mongo_creds_from_config()
client = mongo_module.create_mongo_client(URI)

# Initialize FastAPI
app = FastAPI()

# Pydantic model for input validation
class ChatRequest(BaseModel):
    user_id: str
    prompt: str

@app.post("/generate")
def generate(request: ChatRequest):
    user_id = request.user_id
    prompt = request.prompt
    timestamp = datetime.utcnow()

    # Prepare the user message with timestamp
    user_message = {"role": "user", "content": prompt, "timestamp": timestamp}

    # Fetch previous chat history from MongoDB
    chat_history = list(mongo_module.get_conversation(client, "personalAI", "chatHistory", user_id))

    # Append the new message
    chat_history.append(user_message)

    # Prepare system message
    system_prompt = {"role": "system", "content": "You are a patient and knowledgeable teacher."}

    # Build message list for LLM
    messages = [system_prompt] + chat_history

    # Call the LLM model
    response = ollama.chat(model="llama3.2:1b", messages=messages)
    model_response = response['message']['content']

    # Store assistant response with timestamp
    assistant_message = {"role": "assistant", "content": model_response, "timestamp": datetime.utcnow()}
    chat_history.append(assistant_message)

    # Store updated chat history in MongoDB
    mongo_module.insert_data(client, "personalAI", "chatHistory", {
        "user_id": user_id,
        "conversation": chat_history
    })

    # Return response including model response, chat history, and timestamp
    return {
        "response": model_response,
        "chat_history": chat_history,
        "timestamp": timestamp  # Explicitly return the timestamp here
    }


@app.get("/history/{user_id}")
def get_chat_history(user_id: str):
    chat_history = mongo_module.get_conversation(client, "personalAI", "chatHistory", user_id)
    # Only keep the last 10 conversations
    chat_history = chat_history[-10:]
    return {"history": chat_history}


@app.post("/reset")
def reset_session(user_id: str):
    result = mongo_module.delete_data(client, "personalAI", "chatHistory", {"user_id": user_id})
    
    if result.deleted_count > 0:
        return {"message": f"Session for {user_id} has been reset."}
    else:
        raise HTTPException(status_code=404, detail="Session not found")

# Apply nest_asyncio for running in Jupyter (only needed if using Jupyter)
nest_asyncio.apply()

# Start FastAPI server (use only when running script directly)
if __name__ == "__main__":
    uvicorn.run(app, host="0.0.0.0", port=8000)


INFO:     Started server process [92962]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


Connected to MongoDB successfully!


/var/folders/cq/hc047wm16ks3ystn97nbx3980000gn/T/ipykernel_92962/665709683.py:26: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow()


Data inserted successfully with ID: 67b26ef452428ec61e7289f8
INFO:     127.0.0.1:54801 - "POST /generate HTTP/1.1" 200 OK


/var/folders/cq/hc047wm16ks3ystn97nbx3980000gn/T/ipykernel_92962/665709683.py:48: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  assistant_message = {"role": "assistant", "content": model_response, "timestamp": datetime.utcnow()}


Data inserted successfully with ID: 67b2702d52428ec61e7289f9
INFO:     127.0.0.1:54880 - "POST /generate HTTP/1.1" 200 OK
Data inserted successfully with ID: 67b2705552428ec61e7289fa
INFO:     127.0.0.1:54900 - "POST /generate HTTP/1.1" 200 OK
Data inserted successfully with ID: 67b2865852428ec61e7289fb
INFO:     127.0.0.1:56232 - "POST /generate HTTP/1.1" 200 OK
Data inserted successfully with ID: 67b286dc52428ec61e7289fc
INFO:     127.0.0.1:56286 - "POST /generate HTTP/1.1" 200 OK
Data inserted successfully with ID: 67b2870152428ec61e7289fd
INFO:     127.0.0.1:56291 - "POST /generate HTTP/1.1" 200 OK
Data inserted successfully with ID: 67b2872352428ec61e7289fe
INFO:     127.0.0.1:56294 - "POST /generate HTTP/1.1" 200 OK
Data inserted successfully with ID: 67b2874952428ec61e7289ff
INFO:     127.0.0.1:56297 - "POST /generate HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [92962]
